# Run eQTL analysis for lupus data

pkill -9 -f joblib

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import itertools
from pybedtools import BedTool
import statsmodels.formula.api as smf
import statsmodels.api as sm
import imp

import os
import pickle as pkl
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.9-py3.8.egg')
sys.path.append('/home/ssm-user/Github/misc-seq/miscseq/')
import encode
import memento
import run_memento

In [3]:
data_path  = '/data_volume/memento/lupus/'

### Get SNP gene pairs from Gracie's output for sanity checking

In [4]:
# gs_pairs_mine = pd.read_csv(data_path +'mateqtl_input/{}/gene_snp_pairs_{}.csv'.format('eur', '100kb'))

In [5]:
# gracie_nk = pd.read_csv(data_path + 'gracie_eqtls/nk_metasoft_out.txt', sep='\t', skiprows=1, header=None).iloc[:, :3]

In [6]:
# gene_snp_pairs_new = pd.DataFrame()
# gene_snp_pairs_new['gene'] = gracie_nk.iloc[:, 0].str.split('_').str[1]
# gene_snp_pairs_new['rsid'] = gracie_nk.iloc[:, 0].str.split('_').str[0]

In [7]:
# gene_snp_pairs_new.to_csv(data_path + 'gracie_eqtls/gene_snp_pairs.csv', index=False)

### Create shuffled genotypes for sanity checks

In [8]:
# for pop in ['asian', 'eur']:
#     pos = pd.read_csv(data_path + 'mateqtl_input/{}_genos.tsv'.format(pop), sep='\t', index_col=0)
#     pos_mtx = pos.values.copy()
#     a = [np.random.shuffle(x) for x in pos_mtx]
#     pd.DataFrame(pos_mtx, columns=pos.columns, index=pos.index).to_csv(data_path + 'mateqtl_input/{}_genos_shuffled.tsv'.format(pop), sep='\t')

### Run memento on full gene-SNP pairs

In [9]:
# # mean
# imp.reload(run_memento)
# for pop in ['asian','eur']:
#     print('---', pop, '---')
#     run_memento.run_full(pop)

In [ ]:
# variability
imp.reload(run_memento)
for pop in ['asian','eur']:
    print('---', pop, '---')
    run_memento.run_full(pop, parameter='variability', num_blocks=5)

--- asian ---
961210
T4


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)


0


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)
[Parallel(n_jobs=80)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=80)]: Done  40 tasks      | elapsed:   10.8s
[Parallel(n_jobs=80)]: Done 290 tasks      | elapsed:   27.1s
[Parallel(n_jobs=80)]: Done 640 tasks      | elapsed:   51.2s
[Parallel(n_jobs=80)]: Done 1090 tasks      | elapsed:  1.4min
[Parallel(n_jobs=80)]: Done 1640 tasks      | elapsed:  2.2min
[Parallel(n_jobs=80)]: Done 2290 tasks      | elapsed:  2.9min


### Combine blocks

In [9]:
for pop in ['asian', 'eur']:
    for ct in ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']:
        result = []
        for block in range(5):
            result.append(pd.read_csv('/data_volume/memento/lupus/memento_full/100kb/{}_{}_block_{}.csv'.format(pop, ct, block)))
        result = pd.concat(result)
        result['FDR'] = memento.util._fdrcorrect(result['de_pval'])
        result['statistic'] = result['de_coef']/result['de_se']
        result = result[['tx', 'gene', 'statistic', 'de_pval', 'FDR','de_coef']]
        result.columns = ["SNP","gene","t-stat","p-value","FDR","beta"]
        print(result.query('FDR < 0.1').shape[0])
        result.to_csv(data_path + 'full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct), index=False)

48863
93693
35330
53100
35401
20379
61242
93087
35766
32052
29423
22323


In [5]:
for pop in ['asian', 'eur']:
    for ct in ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']:
        result = []
        for block in range(5):
            result.append(pd.read_csv('/data_volume/memento/lupus/memento_full/100kb/{}_{}_block_{}_variability.csv'.format(pop, ct, block)))
        result = pd.concat(result)
        result['de_fdr'] = memento.util._fdrcorrect(result['de_pval'])
        result['dv_fdr'] = memento.util._fdrcorrect(result['dv_pval'])
        result = result[['tx', 'gene', 'dv_coef', 'de_coef', 'de_pval', 'de_fdr', 'dv_pval','dv_fdr']]
        print(result.query('dv_fdr < 0.1').gene.drop_duplicates().shape[0])
        result.to_csv(data_path + 'full_analysis/memento/100kb/{}_{}_variability.csv'.format(pop, ct), index=False)

30
63
335
64
51
46
8
45
115
48
41
45


In [ ]:
for pop in ['asian', 'eur']:
    for ct in ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']:
        result = []
        for block in range(5):
            result.append(pd.read_csv('/data_volume/memento/lupus/memento_full/100kb/{}_{}_block_{}_variability_no_approx.csv'.format(pop, ct, block)))
        result = pd.concat(result)
        result['de_fdr'] = memento.util._fdrcorrect(result['de_pval'])
        result['dv_fdr'] = memento.util._fdrcorrect(result['dv_pval'])
        result = result[['tx', 'gene', 'dv_coef', 'de_coef', 'de_pval', 'de_fdr', 'dv_pval','dv_fdr']]
        print(result.query('dv_fdr < 0.1').gene.drop_duplicates().shape[0])
        result.to_csv(data_path + 'full_analysis/memento/100kb/{}_{}_variability_no_approx.csv'.format(pop, ct), index=False)

### Select genes from pseudobulk for Matrix eQTL

In [2]:
cts = ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']

In [7]:
for pop in ['asian', 'eur']:
    for ct in cts:
        print(pop, ct)
        results = pd.read_csv(data_path + 'full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct))

        pseudobulk = pd.read_csv(data_path + 'pseudobulk/{}_{}.csv'.format(pop, ct), index_col=0, sep='\t')
        pseudobulk = pseudobulk.loc[:, results['gene'].drop_duplicates().tolist()]
        pseudobulk.T.to_csv(data_path + 'full_analysis/mateqtl/pseudobulk/{}_{}.csv'.format(pop, ct), sep='\t')

asian T4
asian cM
asian ncM
asian T8
asian B
asian NK
eur T4
eur cM
eur ncM
eur T8
eur B
eur NK


### Save memento eQTLs

In [5]:
eqtls = []
for pop in ['asian', 'eur']:
    for ct in ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']:
        
        df = pd.read_csv(data_path + 'full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct)).query('FDR < 0.05')
        df['pop'] = pop
        df['ct'] = ct
        eqtls.append(df)
eqtls = pd.concat(eqtls)        

In [6]:
eqtls.to_csv(data_path + '../tables/Supplementary_Table_5_SLE_eQTL.csv', index=False)